In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

from imblearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import BayesianRidge
from sklearn.naive_bayes import GaussianNB

In [1]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

X_train_pre = df_train.drop(['target', 'id'], axis=1)
y = df_train['target']

X_test_pre = df_test.drop('id', axis=1)

NameError: name 'pd' is not defined

In [61]:
for i, elem in enumerate(X_test_pre['f0']):
    if X_test_pre['f15'][i] != elem:
        print(i)
        if elem != np.nan and X_test_pre['f15'][i] != np.nan:
            print(f'f0: {elem}, f15: {X_test_pre["f15"][i]}')

8
f0: nan, f15: nan
16
f0: nan, f15: nan
67
f0: nan, f15: nan
112
f0: nan, f15: nan
160
f0: nan, f15: nan
219
f0: nan, f15: nan
234
f0: nan, f15: nan
249
f0: nan, f15: nan
268
f0: nan, f15: nan
273
f0: nan, f15: nan
275
f0: nan, f15: nan
293
f0: nan, f15: nan
314
f0: nan, f15: nan
344
f0: nan, f15: nan
368
f0: nan, f15: nan
422
f0: nan, f15: nan
445
f0: nan, f15: nan
486
f0: nan, f15: nan
530
f0: nan, f15: nan
553
f0: nan, f15: nan
592
f0: nan, f15: nan
716
f0: nan, f15: nan
822
f0: nan, f15: nan
905
f0: nan, f15: nan
989
f0: nan, f15: nan
1008
f0: nan, f15: nan
1015
f0: nan, f15: nan
1022
f0: nan, f15: nan
1090
f0: nan, f15: nan
1138
f0: nan, f15: nan
1141
f0: nan, f15: nan
1142
f0: nan, f15: nan
1196
f0: nan, f15: nan
1217
f0: nan, f15: nan
1220
f0: nan, f15: nan
1228
f0: nan, f15: nan
1254
f0: nan, f15: nan
1332
f0: nan, f15: nan
1389
f0: nan, f15: nan
1426
f0: nan, f15: nan
1429
f0: nan, f15: nan
1444
f0: nan, f15: nan
1503
f0: nan, f15: nan
1529
f0: nan, f15: nan
1586
f0: nan, f15

16517
f0: nan, f15: nan
16562
f0: nan, f15: nan
16565
f0: nan, f15: nan
16592
f0: nan, f15: nan
16593
f0: nan, f15: nan
16606
f0: nan, f15: nan
16607
f0: nan, f15: nan
16690
f0: nan, f15: nan
16730
f0: nan, f15: nan
16738
f0: nan, f15: nan
16937
f0: nan, f15: nan
16998
f0: nan, f15: nan
17009
f0: nan, f15: nan
17033
f0: nan, f15: nan
17073
f0: nan, f15: nan
17155
f0: nan, f15: nan
17168
f0: nan, f15: nan
17233
f0: nan, f15: nan
17251
f0: nan, f15: nan
17264
f0: nan, f15: nan
17291
f0: nan, f15: nan
17346
f0: nan, f15: nan
17364
f0: nan, f15: nan
17371
f0: nan, f15: nan
17376
f0: nan, f15: nan
17379
f0: nan, f15: nan
17380
f0: nan, f15: nan
17388
f0: nan, f15: nan
17447
f0: nan, f15: nan
17545
f0: nan, f15: nan
17586
f0: nan, f15: nan
17597
f0: nan, f15: nan
17612
f0: nan, f15: nan
17613
f0: nan, f15: nan
17668
f0: nan, f15: nan
17671
f0: nan, f15: nan
17700
f0: nan, f15: nan
17707
f0: nan, f15: nan
17771
f0: nan, f15: nan
17805
f0: nan, f15: nan
17825
f0: nan, f15: nan
17964
f0: nan, f

29399
f0: nan, f15: nan
29422
f0: nan, f15: nan
29446
f0: nan, f15: nan
29528
f0: nan, f15: nan
29576
f0: nan, f15: nan
29579
f0: nan, f15: nan
29665
f0: nan, f15: nan
29739
f0: nan, f15: nan
29755
f0: nan, f15: nan
29780
f0: nan, f15: nan
29793
f0: nan, f15: nan
29794
f0: nan, f15: nan
29796
f0: nan, f15: nan
29803
f0: nan, f15: nan
29809
f0: nan, f15: nan
29831
f0: nan, f15: nan
29885
f0: nan, f15: nan
29947
f0: nan, f15: nan
29948
f0: nan, f15: nan
29966
f0: nan, f15: nan
30003
f0: nan, f15: nan
30036
f0: nan, f15: nan
30080
f0: nan, f15: nan
30091
f0: nan, f15: nan
30102
f0: nan, f15: nan
30118
f0: nan, f15: nan
30121
f0: nan, f15: nan
30191
f0: nan, f15: nan
30220
f0: nan, f15: nan
30221
f0: nan, f15: nan
30229
f0: nan, f15: nan
30244
f0: nan, f15: nan
30253
f0: nan, f15: nan
30319
f0: nan, f15: nan
30329
f0: nan, f15: nan
30353
f0: nan, f15: nan
30362
f0: nan, f15: nan
30381
f0: nan, f15: nan
30403
f0: nan, f15: nan
30406
f0: nan, f15: nan
30413
f0: nan, f15: nan
30423
f0: nan, f

41121
f0: nan, f15: nan
41187
f0: nan, f15: nan
41209
f0: nan, f15: nan
41262
f0: nan, f15: nan
41296
f0: nan, f15: nan
41318
f0: nan, f15: nan
41324
f0: nan, f15: nan
41357
f0: nan, f15: nan
41369
f0: nan, f15: nan
41402
f0: nan, f15: nan
41433
f0: nan, f15: nan
41467
f0: nan, f15: nan
41490
f0: nan, f15: nan
41497
f0: nan, f15: nan
41612
f0: nan, f15: nan
41617
f0: nan, f15: nan
41622
f0: nan, f15: nan
41677
f0: nan, f15: nan
41691
f0: nan, f15: nan
41826
f0: nan, f15: nan
41830
f0: nan, f15: nan
41852
f0: nan, f15: nan
41903
f0: nan, f15: nan
41906
f0: nan, f15: nan
41922
f0: nan, f15: nan
41954
f0: nan, f15: nan
41994
f0: nan, f15: nan
41996
f0: nan, f15: nan
42011
f0: nan, f15: nan
42102
f0: nan, f15: nan
42108
f0: nan, f15: nan
42122
f0: nan, f15: nan
42130
f0: nan, f15: nan
42216
f0: nan, f15: nan
42237
f0: nan, f15: nan
42240
f0: nan, f15: nan
42291
f0: nan, f15: nan
42343
f0: nan, f15: nan
42360
f0: nan, f15: nan
42385
f0: nan, f15: nan
42420
f0: nan, f15: nan
42424
f0: nan, f

In [68]:
X_test_pre.loc[(X_test_pre['f0'].ne(X_test_pre['f15'])) & (X_test_pre['f0'].isnull() & ~X_test_pre['f15'].isnull())]

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28
20956,NaN,DE,6caf9a70e,540b0e83b,6.8770,A,NaN,NaN,2.0,TH,1.0,TH,f,1a59581be,T,O,5.0,0.3,6b56b4325,5.0,76b3fb412,3.0,1.0,E,S,5.0,6b56b4325,h,1.0
21034,NaN,IT,71cf45329,77cee33ff,6.9033,A,0.0,R,1.0,gt,0.0,gt,a,44f8a6061,T,T,8.0,0.5,934155e05,3.0,fc5c1c083,2.0,1.0,A,S,104.0,9.34E+10,h,0.0


In [8]:
X_train_pre['f28'].unique()

array([ 1.,  0., nan])

In [51]:
X_test_pre.loc[26648, 'f9'] = 'heihei'
X_test_pre['f9'][26648]

'heihei'

# Inspection of data

In [ ]:
from sklearn.model_selection import train_test_split
X_train_pre, X_test_pre, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y) 

In [ ]:
y_train.head()

In [ ]:
X_train_pre.head()

In [ ]:
for elem in X:
    print(f'Column \'{elem}\': \n{X[elem].unique()}')
    print()

After inspecting each column, it looks as though column f0 and f15 are the same and f9 and f11 are the same, which also seems to be the case (see below.)

Columns f28 and f22 contains only binary values. 

Columns f2, f3, f13, f18, f20 and f26 contains hexadecimal values

Column f28 contains only binary values. 

Column f27 contains the letters from a-m.

Column f26 contains only hexadecimal values

Column f25 contains the values 1-104

Column f24 contains only "N" and "S" (North, south?)

Column f23 contains only A-F (grades?)

Column f22 contains only binary values. 

Column f21 contains the values 1-3

Column f20 contains only hexadecimal values

Column f19 contains the values 1-7

Column f18 contains only hexadecimal values

Column f17 contains the values .1 - .6

Column f16 contains the values 1-12

Column f15 contains the alphabet

Column f14 contains only T and F, which _could_ be True/False.

Column f13 contains only hexadecimal values

Column f12 contains the letters a-f (grades?) NOT THE SAME AS F23

Column f11 contains strings with two letters

Column f10 contains only the values 0, 1, 10 and 11

Column f9 contains strings with two letters

Column f8 contains the values 1-6

Column f7 contains only R, G, B (which might be Red, Green, Blue?)

Column f6 contains only the values 0-2

Column f5 contains only "A" and "B"

Column f4 contains floats

Column f3 contains only hexadecimal values

Column f2 contains only hexadecimal values

Column f1 contains only 'AU', 'CN', 'DE', 'IT', 'JP'

Column f0 contains the alphabet

In [ ]:
print('F0 == F15?', end=' ')
print(X['f0'].equals(X['f15']))

print('F9 == F11?', end=' ')
print(X['f9'].equals(X['f11']))

print('F9 == F11?', end=' ')
print(X['f12'].dropna().equals(X['f23'].dropna().apply(lambda x:x.lower())))

In [ ]:
np.sort(X['id'].dropna().unique())

In [ ]:
hex_columns = ['f2', 'f3', 'f13', 'f18', 'f20', 'f26']
bool_columns = ['f28', 'f22', 'f14']
ordinal_columns = ['f6', 'f8', 'f16', 'f17', 'f19', 'f21', 'f25']
categorical_columns = ['f1', 'f5', 'f7', 'f9', 'f11', 'f12', 'f23', 'f24', 'f27']
numerical_columns = ['f4']
alphabet_columns = ['f0', 'f15']

# Duplicates

In [ ]:
X.duplicated().any()

# Missing values

In [ ]:
round(100*(X.isnull().sum()/len(X.index)),2)

We can see that there are ~3% missing values in each column. Missing values can be handeled in many different ways, one is to drop all rows which contains missing values. This can clean up the data, but at the same time it reduces the little data we already have. Since there are only ~3% we firstly try to drop the rows, and see how much data we will miss:

In [ ]:
dropped_nan = X.dropna()
print(f'Dataset percent after dropping nan-rows: {(dropped_nan.shape[0] / X.shape[0]) * 100}')
print(f'Rows removed: {X.shape[0] - dropped_nan.shape[0]}')

We see that by dropping all rows with missing values, we are removing ~54% of the entire data. This is unacceptable, so we have to do something else with the missing rows. One common approach is to set the missing values as the mean of the whole column. We'll try to do this:

In [ ]:
def conv_hex(x):
    try:
        return int(x, 16)
    except ValueError as e:
        return np.nan
    except TypeError as e:
        return np.nan
    
def conv_bool(x):
    try:
        if not type(x) == str:
            return x
        if x.lower() == 'f':
            return 0
        elif x.lower() == 't':
            return 1
        return x
    except e:
        return np.nan

def preprocess(df):
    for col in hex_columns:
        col_loc = list(df.columns).index(col)
        # print(df.iloc[:,col_loc])
        df.iloc[:,col_loc] = df[col].apply(lambda x: conv_hex(x)).values

    for col in bool_columns:
        col_loc = list(df.columns).index(col)
        df.iloc[:,col_loc] = df[col].apply(lambda x: conv_bool(x)).values

    for col in numerical_columns + ordinal_columns:
        df[col].fillna(np.mean(X[col]))

preprocess(X_train_pre)
preprocess(X_test_pre)
# for elem in X:
#     print(f'Column \'{elem}\': \n{X[elem].unique()}')
#     print()

In [ ]:
X.shape

In [ ]:
numeric_transformer = make_pipeline(
    IterativeImputer(random_state=42),
    StandardScaler()
)

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore')
)
    
preprocessor = make_column_transformer( 
    (numeric_transformer, numerical_columns + hex_columns + bool_columns + ordinal_columns),
    (categorical_transformer, categorical_columns + alphabet_columns),
)

print(X_test_pre.shape)

X_train = pd.DataFrame(preprocessor.fit_transform(X_train_pre, y_train).toarray())
X_test  = pd.DataFrame(preprocessor.transform(X_test_pre).toarray())

# X_train.describe()

In [ ]:
X_train.isna().any().unique()

# Prediction

In [ ]:
X_test.shape

In [ ]:
%%time

# lgbm = LGBMClassifier(random_state=10, learning_rate=.125, num_leaves=20000, is_unbalance=True)

gnb = GaussianNB(priors = [.9, .1])

# lgbm.fit(X_train, y_train)
gnb.fit(X = X_train, y = y_train)

# y_pred = lgbm.predict(X_test)

y_pred = gnb.predict_proba(X_test)

for elem in y_pred:
    print(elem)

In [ ]:
y_pred = [np.argmax(i) for i in y_pred]

In [ ]:
pred_df = pd.DataFrame(y_pred)

keys = [i for i in range(50000, 100000)]

# pred_df.set_index(keys)
pred_df.insert(0, "id", keys)
pred_df.rename(columns={0: 'target'}, inplace=True)

pred_df.to_csv('./predictions5.csv', index=False)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_pred, y_test)))

In [ ]:
ones = 0
zeros = 0
for elem in y_pred:
    if elem == 1:
        ones += 1
    elif elem == 0:
        zeros += 1

print(f'Ones: {ones}, Zeros: {zeros}')
print(f'Zeros vs ones: {1 - ones/zeros}')